In [32]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import optimizers
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [86]:
filename = "pass_new.txt"
raw_text = open(filename,encoding="utf8").read()
raw_text = raw_text

In [87]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [88]:
char_to_int

{'\n': 0,
 '!': 1,
 '"': 2,
 '#': 3,
 '$': 4,
 '%': 5,
 '&': 6,
 '(': 7,
 ')': 8,
 '*': 9,
 '+': 10,
 '-': 11,
 '.': 12,
 '/': 13,
 '0': 14,
 '1': 15,
 '2': 16,
 '3': 17,
 '4': 18,
 '5': 19,
 '6': 20,
 '7': 21,
 '8': 22,
 '9': 23,
 '<': 24,
 '=': 25,
 '>': 26,
 '?': 27,
 '@': 28,
 'A': 29,
 'B': 30,
 'C': 31,
 'D': 32,
 'E': 33,
 'F': 34,
 'G': 35,
 'H': 36,
 'I': 37,
 'J': 38,
 'K': 39,
 'L': 40,
 'M': 41,
 'N': 42,
 'O': 43,
 'P': 44,
 'Q': 45,
 'R': 46,
 'S': 47,
 'T': 48,
 'U': 49,
 'V': 50,
 'W': 51,
 'X': 52,
 'Y': 53,
 'Z': 54,
 '\\': 55,
 '^': 56,
 '_': 57,
 '`': 58,
 'a': 59,
 'b': 60,
 'c': 61,
 'd': 62,
 'e': 63,
 'f': 64,
 'g': 65,
 'h': 66,
 'i': 67,
 'j': 68,
 'k': 69,
 'l': 70,
 'm': 71,
 'n': 72,
 'o': 73,
 'p': 74,
 'q': 75,
 'r': 76,
 's': 77,
 't': 78,
 'u': 79,
 'v': 80,
 'w': 81,
 'x': 82,
 'y': 83,
 'z': 84}

In [89]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  954253
Total Vocab:  85


In [90]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 25
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  954228


In [91]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [92]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [93]:
X.shape

(954228, 25, 1)

In [95]:
# define the checkpoint
filepath="Bigger_Model_weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X, y, epochs=5, batch_size=128, callbacks=callbacks_list)

Epoch 1/5
503936/954228 [==============>...............] - ETA: 6:47 - loss: 3.1001

## Generating the Passwords from the Trained Model

In [46]:
# load the network weights
filename = "weights-improvement-05-3.0342.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [47]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [83]:
userinput = "joshuagee wcfields rainer"

sort_sen = sorted(list(userinput))
pattern = [char_to_int[value.lower()] for value in userinput]

In [84]:
len(pattern)

25

In [85]:
import sys
# pick a random seed
#start = numpy.random.randint(0,100)
#pattern = dataX[start]
print ("Seed:")

print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" joshuagee wcfields rainer "
12
123456
saneea12
123456
saneea12
123456
saneea12
123456
saneea12
123456
saneea12
123456
saneea12
123456
saneea12
123456
saneea12
123456
saneea12
123456
saneea12
123456
saneea12
123456
saneea12
123456
saneea12
123456
saneea12
123456
saneea12
123456
saneea12
123456
saneea12
123456
saneea12
123456
saneea12
123456
sa

KeyboardInterrupt: 

In [69]:
len(pattern)

25